# Arbre de décision
Implémentation d'un arbre de décision avec une recherche d'hyper-paramètres.
<br>
### Etapes

1. **Chargement des données** à partir des dossiers déjà splités (train/test).
2. **Pré-traîtement** des données. 
3. **Initialisation du modèle**.
4. **Recherche d'hyper-paramètres**.
5. **Evaluation du modèle** et **prédiction**.

<br>

### Conclusion

- Grande différence entre le score de validation croisée et le score sur les données de tests (+15%).
- Il n'est pas forcément nécessaire de normaliser les données dans le cadre d'un arbre de décision.
- Le modèle est probablement overfitté.

## Chargement et split des données en fonction du dossier

Les images sont redimensionnées à 64x64 pixels pour avoir une taille uniforme et plus légère.

In [1]:
import numpy as np

from src.preprocessing import load_images_from_folder
import os

base_path = os.path.abspath(os.path.join(os.getcwd(), "../..//data/raw"))

train_path = os.path.join(base_path, "train")
test_path = os.path.join(base_path, "test")

X_train, y_train = load_images_from_folder(train_path, image_size=(64, 64))
X_test, y_test = load_images_from_folder(test_path, image_size=(64, 64))

## Pré-traitement des données

- Augmentation des données pour avoir une distribution des classes homogènes
- Normalisation des images pour "rescaler" les valeurs des pixels dans une plage uniforme.
- Aplanissement des images (2D -> 1D) pour les rendre compatibles.

In [3]:
from src.augment import augment_images_to_balance

X_train_augmented, y_train_augmented = augment_images_to_balance(
    X_train, y_train
)

In [11]:
flattened_images = np.array([x.flatten() for x in X_train_augmented])
test_flattened_images = np.array([x.flatten() for x in X_test])

## Initialisation et recherche des hyperparamètres optimaux du modèle avec les données traitées

- Initialisation du modèle basé sur un arbre de décision.
- Définition de la grille de recherche d'hyper-paramètres.
- Recherche d'hyper-paramètres avec GridSearchCV basée sur la validation croisée.
- Entraînement et recherche d'hyper-paramètres

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()

param_grid = {
    'criterion': ['gini'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [None, 'sqrt', 'log2']
}

grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(flattened_images, y_train_augmented)

#### Affichage des hyper-paramètres calculées par la recherche.

In [17]:
grid_search.best_params_

{'ccp_alpha': 0.001,
 'criterion': 'gini',
 'max_depth': 30,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 5}

## Evaluation du meilleur modèle selectionné

- Validation croisée pour évaluer les performance du modèle.
- Prédiction et évaluation du modèle sur les données de test.

In [18]:
from sklearn.model_selection import cross_val_score

best_clf = grid_search.best_estimator_
scores = cross_val_score(best_clf, flattened_images, y_train_augmented, cv=5, scoring='accuracy')
print(f"Scores de validation croisée : {scores}")
print(f"Score moyen de validation croisée : {np.mean(scores)}")

y_pred = best_clf.predict(test_flattened_images)

from sklearn.metrics import classification_report, accuracy_score

print(f"Accuracy sur les données de test : {accuracy_score(y_test, y_pred)}")
print(f"Rapport de classification :\n {classification_report(y_test, y_pred)}")

Scores de validation croisée : [0.8        0.83483871 0.94387097 0.92709677 0.93806452]
Score moyen de validation croisée : 0.8887741935483872
Accuracy sur les données de test : 0.75625
Rapport de classification :
               precision    recall  f1-score   support

           0       0.83      0.45      0.58       242
           1       0.74      0.94      0.83       398

    accuracy                           0.76       640
   macro avg       0.78      0.70      0.71       640
weighted avg       0.77      0.76      0.74       640
